# Exploratory Data Analysis #1

### Import libraries

In [2]:
import pandas as pd
from surprise import Reader, Dataset

### Import data

In [54]:
df2 = pd.read_csv('comb_df7.csv')
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54387 entries, 0 to 54386
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   genre             54387 non-null  object 
 1   title             54387 non-null  object 
 2   author            54387 non-null  object 
 3   pages             54387 non-null  int64  
 4   average_rating    54387 non-null  float64
 5   book_description  40773 non-null  object 
 6   user_unique_key   54387 non-null  int64  
 7   number_of_stars   54387 non-null  int64  
 8   book_unique_key   54387 non-null  int64  
 9   review_text       53340 non-null  object 
dtypes: float64(1), int64(4), object(5)
memory usage: 4.1+ MB


In [5]:
reader = Reader()
data = Dataset.load_from_df(df2,reader)

ValueError: too many values to unpack (expected 3)

In [31]:
new_df = df2.drop('genre','title','author', 'pages', 'average_rating', 'book_description', 'review_text')

In [32]:
reader = Reader()
data = Dataset.load_from_df(new_df,reader)

AttributeError: 'DataFrame' object has no attribute 'itertuples'

### Inspect the data

### Pre-process the data

In [ ]:
#join the tables

In [ ]:
#check for nulls/nans

### EDA visualizations

### Build first simple model (FSM)

### FSM visualizations

In [7]:

from pyspark.sql import SparkSession
from pyspark.ml.evaluation import RegressionEvaluator

from pyspark.ml.recommendation import ALS

from pyspark.ml.evaluation import RegressionEvaluator

from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
import pyspark
spark = SparkSession.builder.getOrCreate()

from pyspark.sql.types import IntegerType
from pyspark.sql.functions import isnan, when, count, col
from pyspark.sql.types import *

In [4]:
import os
import pandas as pd

In [51]:
# dfpd= pd.read_csv('comb_df7.csv')

In [47]:
dfpd['user_unique_key'].nunique()

27642

In [39]:
#df2 = spark.read.csv('comb_df7.csv', header='true', inferSchema='true')

In [27]:
df2 = spark.read.csv('comb_df7.tsv', header ='true', sep=r"\t")

In [56]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54387 entries, 0 to 54386
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   genre             54387 non-null  object 
 1   title             54387 non-null  object 
 2   author            54387 non-null  object 
 3   pages             54387 non-null  int64  
 4   average_rating    54387 non-null  float64
 5   book_description  40773 non-null  object 
 6   user_unique_key   54387 non-null  int64  
 7   number_of_stars   54387 non-null  int64  
 8   book_unique_key   54387 non-null  int64  
 9   review_text       53340 non-null  object 
dtypes: float64(1), int64(4), object(5)
memory usage: 4.1+ MB


In [64]:
dfpd= pd.read_csv('comb_df7.csv')

In [68]:
from pyspark.sql.types import *

schema = StructType([StructField("genre", StringType(), True),StructField("title", StringType(), True), 
                     StructField("author", StringType(), True), StructField("pages", IntegerType(), True), 
                     StructField("average_rating", FloatType(), True), StructField("book_description", StringType(), True), 
                     StructField("user_unique_key", IntegerType(), True),StructField("number_of_stars", IntegerType(), True), 
                     StructField("book_unique_key", IntegerType(), True), StructField("review_text", StringType(), True)])

sp_df = spark.createDataFrame(dfpd, schema=schema)

In [69]:
sp_df.dtypes

[('genre', 'string'),
 ('title', 'string'),
 ('author', 'string'),
 ('pages', 'int'),
 ('average_rating', 'float'),
 ('book_description', 'string'),
 ('user_unique_key', 'int'),
 ('number_of_stars', 'int'),
 ('book_unique_key', 'int'),
 ('review_text', 'string')]

In [5]:
sm_df= pd.read_csv('small_df.csv')

In [8]:
schema = StructType([StructField("genre", StringType(), True),StructField("title", StringType(), True), 
                     StructField("author", StringType(), True), StructField("pages", IntegerType(), True), 
                     StructField("average_rating", FloatType(), True), StructField("book_description", StringType(), True), 
                     StructField("user_unique_key", IntegerType(), True),StructField("number_of_stars", IntegerType(), True), 
                     StructField("book_unique_key", IntegerType(), True), StructField("review_text", StringType(), True)])

df_sm2 = spark.createDataFrame(sm_df, schema=schema)

In [ ]:
# pddf = pd.read_csv("comb_df7.csv")
# sp_df = spark.createDataFrame(pd_df, schema=schema)

In [ ]:
df2=dfpd= pd.read_csv('comb_df7.csv')

In [40]:
df2.dtypes

[('genre', 'string'),
 ('title', 'string'),
 ('author', 'string'),
 ('pages', 'string'),
 ('average_rating', 'string'),
 ('book_description', 'string'),
 ('user_unique_key', 'string'),
 ('number_of_stars', 'string'),
 ('book_unique_key', 'string'),
 ('review_text', 'string')]

In [41]:
output_df = df2.withColumn("number_of_stars",df2["number_of_stars"].cast(IntegerType()))

In [42]:
output_df.dtypes

[('genre', 'string'),
 ('title', 'string'),
 ('author', 'string'),
 ('pages', 'string'),
 ('average_rating', 'string'),
 ('book_description', 'string'),
 ('user_unique_key', 'string'),
 ('number_of_stars', 'int'),
 ('book_unique_key', 'string'),
 ('review_text', 'string')]

In [43]:
output2 = output_df.withColumn('book_unique_key', output_df['book_unique_key'].cast(IntegerType()))

In [44]:
output2.dtypes

[('genre', 'string'),
 ('title', 'string'),
 ('author', 'string'),
 ('pages', 'string'),
 ('average_rating', 'string'),
 ('book_description', 'string'),
 ('user_unique_key', 'string'),
 ('number_of_stars', 'int'),
 ('book_unique_key', 'int'),
 ('review_text', 'string')]

In [45]:
out3= output2.withColumn('user_unique_key', output2['user_unique_key'].cast(IntegerType()))

In [46]:
out3.dtypes

[('genre', 'string'),
 ('title', 'string'),
 ('author', 'string'),
 ('pages', 'string'),
 ('average_rating', 'string'),
 ('book_description', 'string'),
 ('user_unique_key', 'int'),
 ('number_of_stars', 'int'),
 ('book_unique_key', 'int'),
 ('review_text', 'string')]

In [47]:
out3.where(out3['user_unique_key'].isNull()).select('user_unique_key','title').show()

+---------------+--------------------+
|user_unique_key|               title|
+---------------+--------------------+
|           null|The Neil Gaiman R...|
|           null| The Neil Gaiman ...|
|           null|          entrancing|
|           null| The Neil Gaiman ...|
|           null|The Neil Gaiman R...|
|           null| The Neil Gaiman ...|
|           null|          entrancing|
|           null| The Neil Gaiman ...|
|           null| and those who ha...|
|           null| I exaggerate. Bu...|
|           null|                null|
|           null|The Neil Gaiman R...|
|           null| The Neil Gaiman ...|
|           null|          entrancing|
|           null| The Neil Gaiman ...|
|           null|The Neil Gaiman R...|
|           null| The Neil Gaiman ...|
|           null|          entrancing|
|           null| The Neil Gaiman ...|
|           null|The Neil Gaiman R...|
+---------------+--------------------+
only showing top 20 rows



In [75]:
out3.printSchema()

root
 |-- genre: string (nullable = true)
 |-- title: string (nullable = true)
 |-- author: string (nullable = true)
 |-- pages: string (nullable = true)
 |-- average_rating: string (nullable = true)
 |-- book_description: string (nullable = true)
 |-- user_unique_key: integer (nullable = true)
 |-- number_of_stars: integer (nullable = true)
 |-- book_unique_key: integer (nullable = true)
 |-- review_text: string (nullable = true)



In [77]:
# out3.summary().show()

In [92]:

print("The book with the fewest ratings: ")
out3.groupBy("book_unique_key").count().select(min("count")).show()

# print("Avg num ratings per book: ")
# out3.groupBy("book_unique_key").count().select(avg("count")).show()


# print("User with the fewest ratings: ")
# out3.groupBy("user_unique_key").count().select(min("count")).show()

# print("Avg num ratings per user: ")
# out3.groupBy("user_unique_key").count().select(avg("count")).show()

The book with the fewest ratings: 


AnalysisException: cannot resolve '`c`' given input columns: [book_unique_key, count];;
'Project ['c]
+- Aggregate [book_unique_key#1780], [book_unique_key#1780, count(1) AS count#3566L]
   +- Project [genre#1749, title#1750, author#1751, pages#1752, average_rating#1753, book_description#1754, cast(user_unique_key#1755 as int) AS user_unique_key#1791, number_of_stars#1769, book_unique_key#1780, review_text#1758]
      +- Project [genre#1749, title#1750, author#1751, pages#1752, average_rating#1753, book_description#1754, user_unique_key#1755, number_of_stars#1769, cast(book_unique_key#1757 as int) AS book_unique_key#1780, review_text#1758]
         +- Project [genre#1749, title#1750, author#1751, pages#1752, average_rating#1753, book_description#1754, user_unique_key#1755, cast(number_of_stars#1756 as int) AS number_of_stars#1769, book_unique_key#1757, review_text#1758]
            +- Relation[genre#1749,title#1750,author#1751,pages#1752,average_rating#1753,book_description#1754,user_u

"sparsity is calculated by the number of cells in a matrix that contain a rating divided by the total number of values that matrix could hold given the number of users and items (movies). In other words, dividing the number of ratings present in the matrix by the product of users and movies in the matrix and subtracting that from 1 will give us the sparsity or the percentage of the ratings matrix that is empty."

In [74]:
# calculating sparcity--

numerator = out3.select("number_of_stars").count()


num_users = out3.select("user_unique_key").distinct().count()
num_books = out3.select("book_unique_key").distinct().count()


denominator = num_users * num_books


sparsity = (1.0 - (numerator *1.0)/denominator)*100
print("The ratings dataframe is ", "%.2f" % sparsity + "% empty.")

The ratings dataframe is  98.25% empty.


In [93]:
numerator = df_sm2.select("number_of_stars").count()


num_users = df_sm2.select("user_unique_key").distinct().count()
num_books = df_sm2.select("book_unique_key").distinct().count()


denominator = num_users * num_books


sparsity = (1.0 - (numerator *1.0)/denominator)*100
print("The ratings dataframe is ", "%.2f" % sparsity + "% empty.")

The ratings dataframe is  99.60% empty.


In [70]:
train, test = sp_df.randomSplit([0.8, 0.2])

In [71]:
als = ALS(maxIter=5,rank=4, 
          userCol='user_unique_key', itemCol='book_unique_key', 
          ratingCol='number_of_stars',
          coldStartStrategy='drop')


model_fsm = als.fit(train)

In [73]:
predictions = model_fsm.transform(test)
evaluator = RegressionEvaluator(metricName='rmse', labelCol='number_of_stars',
                                predictionCol='prediction')
rmse = evaluator.evaluate(predictions)
print('Root-mean-square error = ' + str(rmse))

Root-mean-square error = 2.0271723460408415


In [87]:

evaluator = RegressionEvaluator(metricName='mae', labelCol='number_of_stars',
                                predictionCol='prediction')
mae = evaluator.evaluate(predictions)
print('mean absolute error = ' + str(mae))

mean absolute error = 1.4594755863126267


In [9]:
train, test = df_sm2.randomSplit([0.8, 0.2])

In [95]:
train, test = df_sm2.randomSplit([0.8, 0.2])
als2 = ALS(maxIter=5,rank=4, 
          userCol='user_unique_key', itemCol='book_unique_key', 
          ratingCol='number_of_stars',
          coldStartStrategy='drop')


model_fsm2 = als2.fit(train)

In [98]:
predictions2 = model_fsm2.transform(test)
evaluator2 = RegressionEvaluator(metricName='rmse', labelCol='number_of_stars',
                                predictionCol='prediction')
rmse2 = evaluator2.evaluate(predictions2)
print('Root-mean-square error = ' + str(rmse2))

Root-mean-square error = 2.0971997522129304


In [99]:
evaluator3 = RegressionEvaluator(metricName='mae', labelCol='number_of_stars',
                                predictionCol='prediction')
mae2 = evaluator3.evaluate(predictions2)
print('mean absolute error = ' + str(mae2))

mean absolute error = 1.4678074366732219


In [10]:
als_model = ALS(userCol='user_unique_key', itemCol='book_unique_key',ratingCol='number_of_stars', 
                rank=10  , maxIter=20 , regParam=.05,
                nonnegative=True,implicitPrefs=False, 
                coldStartStrategy='drop')
model4 = als_model.fit(train)
pred = model4.transform(test)
pred.show()

Py4JJavaError: An error occurred while calling o142.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 11 in stage 140.0 failed 1 times, most recent failure: Lost task 11.0 in stage 140.0 (TID 520, 10.0.0.8, executor driver): java.lang.StackOverflowError
	at java.io.ObjectInputStream.readSerialData(ObjectInputStream.java:2329)
	at java.io.ObjectInputStream.readOrdinaryObject(ObjectInputStream.java:2187)
	at java.io.ObjectInputStream.readObject0(ObjectInputStream.java:1667)
	at java.io.ObjectInputStream.defaultReadFields(ObjectInputStream.java:2405)
	at java.io.ObjectInputStream.readSerialData(ObjectInputStream.java:2329)
	at java.io.ObjectInputStream.readOrdinaryObject(ObjectInputStream.java:2187)
	at java.io.ObjectInputStream.readObject0(ObjectInputStream.java:1667)
	at java.io.ObjectInputStream.readObject(ObjectInputStream.java:503)
	at java.io.ObjectInputStream.readObject(ObjectInputStream.java:461)
	at scala.collection.immutable.List$Serialization

----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 49507)
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.8/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/opt/anaconda3/lib/python3.8/socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "/opt/anaconda3/lib/python3.8/socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/opt/anaconda3/lib/python3.8/socketserver.py", line 720, in __init__
    self.handle()
  File "/opt/anaconda3/lib/python3.8/site-packages/pyspark/accumulators.py", line 268, in handle
    poll(accum_updates)
  File "/opt/anaconda3/lib/python3.8/site-packages/pyspark/accumulators.py", line 241, in poll
    if func():
  File "/opt/anaconda3/lib/python3.8/site-packages/pyspark/accumulators.py", line 245, in accum_updates
  

In [ ]:
als_model = ALS(userCol='user_unique_key', itemCol='book_unique_key',ratingCol='number_of_stars', 
                rank=  , maxIter=20 , regParam=.05,
                nonnegative=True,implicitPrefs=False, 
                coldStartStrategy='drop')
model4 = als_model.fit(train)
pred = model.transform(test)
                
params = ParamGridBuilder()\
          .addGrid(als_model.regParam, [0.01, 0.001])\
          .addGrid(als_model.rank, [4, 10]).build()


cv = CrossValidator(estimator=als_model, estimatorParamMaps=params,evaluator=evaluator,parallelism=4)
best_model = cv.fit(df2)    

best_model.bestModel.rank

Does it work? do the recommendations make sense?

"when ALS generates recommendations, they are provided in descending order by userId, so there's no need to sort the dataframe."

In [ ]:
origin_rating =
recommended=
print("User #25 Ratings:")
origin_ratings.filter(col("user_unique_key") == 60).sort("number_of_stars", ascending= False).show()


print("User 300 Recommendations:")
recommended.filter(col("user_unique_key") == 300).show()

